### Author: Jose Miguel Bautista
### Updated: 05/31/2024

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import itertools

# Preface

We have gone through the previous material with the aim of finding *optimal* solutions and algorithms.  
There will be times where this is not possible, either theoretically or practically.  
For example, recall that a BFS search of Rubik's cube configurations is memory-constrained to depth 10, but the full configuration space extends to depth 20.  
Later on in the final topic, we will also see there are some problems which are objectively and quantifiably difficult, even for computers.  
Other times, we want to know our solution is optimal beyond a shadow of a doubt.  

## Combinatorial Problems 

Many difficult problems are difficult because they grow as a combinatorial.  
That is to say, the problem may be expressed as "find the right combination of $N$ items. "  
Some examples:
- Traveling Salesman Problem: given a set of destinations to visit, what is the loop that visits all in the least amount of fuel?  
- Knapsack Problem: given a set of items with weights and values, knapsack of fixed capacity, determine which items to keep s.t. total value is maximized.
    - How to efficiently manage space at a warehouse
    - How to make a vehicle to operating specification within manufacturing/budget constraints
- Assignment Problem: given 2 sets (nominally, agents and tasks), what assignment of one to the other maximizes/minimizes some objective/utility function. 
    - [Tenant Mix Problem](https://ecommons.cornell.edu/server/api/core/bitstreams/c540bf58-39ae-4cb7-99a0-d645d36f3b77/content): given a set of empty lots, how does one select potential vendors s.t. they generate (say) the most income for a mall?  

Skiena (rightfully) points out that modern computers are pretty fast, so the best option may just be "simple" exhaustive search.  
As a ballpark figure, $O(10^6)$ search items/second is a reasonable rate (plus or minus a few orders of magnitude for your preferred programming languages).  
For reference, $10^6$ is approximately the number of permutations of $10$ objects, or subsets for $20$ objects. 


# Backtracking 

For any exhaustive search of a combinatorial problem, we want to be systematic.  
To ensure correctness, we want to generate all possible (or at least plausible) solutions.   
To ensure efficiency, we want to generate these solutions at most once.   

*Backtracking* is a systematic way to find all solutions, that, while non-specific, is guaranteed to at least find the correct answer.  
Backtracking is exactly what it sounds like, and you should be familiar with it if you have ever tried to solve:  
- [Mazes](https://en.wikipedia.org/wiki/Ariadne%27s_thread_(logic)) / Platformers
- Solitaire
- Sudoku 
- Crosswords

If you have never played any of these, add one to your homework (it will not be graded).  
Otherwise, a backtracking solution can be understood as DFS on an *implicit* graph.  

Consider a labyrinth trapping a [prisoner](https://en.wikipedia.org/wiki/Theseus#Theseus_and_the_Minotaur) who wants to find something (exit, treasure, etc.).  
As before, this labyrinth has an associated graph structure.  
But the prisoner can only see directly around them, so they doesn't know the full structure of the tree (only the local structure).  
They can at least remember everything they have seen, so the tree can be built as they traverse the labyrinth.  
From the prisoner's perspective, the most energy-efficient strategy is to 
1. Go forward as far as possible.  
1. IF:
    1. at the destination, you are done. 
    1. at a fork, make an arbitrary choice among any unexplored paths.  
    1. at a dead-end, back up to a fork with unexplored paths. 

<img src="img/17_labyrinth.png" style="width: 30em" />  

This is equivalent to DFS exploring nodes in order of increasing distance from the root node.  
To clarify, I am *not* saying you should reach for graph structures to solve every backtracking problem.  
What I am saying is that many of the reasons for why something is done can be understood in terms of this tree-based search idea.  

More generally, backtracking can be thought of as constructing a solution represented by a vector $\vec{a} = (a_1, a_2, \ldots, a_k)$.  
At any iteration, we try to extend potential solutions by adding another valid entry $\vec{a} \rightarrow (a_1, a_2, \ldots, a_k, a_{k+1})$ and check the result.  
If the solution is terminal, we are done and terminate the run.  
If the solution is non-terminal, recur and extend again.  
If no extensions are possible, undo the extensions up to the last arbitrary choice and make a different extention.  

For better or worse, this is the limit of the theory behind backtracking alone, at least as far as we care.  
So the rest of this notebook will rely upon examples and exercises.  

# Example - Chess (Permutations) 

**Chess** is a 2-player game on an $8\times 8$ board.  
Players control pieces and pawns, each of which attacks a number of squares around it, and the goal is to trap the enemy king (checkmate).  
For our purposes, we will only need to know 2 pieces: the rook and the queen.  

*Rooks* are pieces which can attack all other squares in the same row ("rank") and column ("file") as its own.  
*Queens* are the same as rooks, except they also attack all diagonal squares as well.  
The *8-rooks problem* is the problem of finding an arrangement of $8$ rooks on a standard $8\times 8$ board s.t. no rooks attack each other.  
More generally, the *$N$-rooks problem* is finding a similar arrangement of $N$ rooks on an $N\times N$ board.  
The *$N$-queens problem* is the same, but with queens instead of rooks.  

## $N$-Rooks
Finding at least one solution to the $N$-rooks problem is fairly easy: each row and column must be occupied by exactly one rook.  
The simplest way to do this is to put all the rooks on the long diagonal, but it's clearly not the only solution.  
How many solutions are there to this problem?  
This is a relatively easy question to answer theoretically: it is all permutations of the column positions, which is $8!$  
For a computer to actually list it out, possibly for other downstream applications, we need to represent the rook positions and permute them.  

<img src="img/17_rookDiag.png" style="width: 30em" />  

**Step 1:** Find a good way to represent the thing we want to permute.  
The most direct method would be to translate rank and file into $N$ tuples.   
But a more convenient way to do it can be found by noting $N$ rooks have to be on $N$ different ranks (or files if you prefer).  
We may as well represent them as an $N$-length array, where the $i^{th}$ index corresponds to the rook on the $i^{th}$ rank, and the entry contains its file.  
For example, I might represent a solution as $(a,b,c,d,e,f,g,h)$, which is all rooks on the long diagonal.  
Equally, I could represent the files by integers, e.g. $(a \rightarrow 1, b \rightarrow 2, \ldots, h \rightarrow 8)$.  
This means we just need to find the permutations of $N$ integers.  

**Step 2:** Find a way to permute the items.  
As a rule, you should first check if there is a library or native method that will do a large part of the job.  
Failing that, you can also check branches on github and course pages.  
If your problem is general enough there's a good chance it exists, and for us we have the `itertools` library.  
**N.b.** Same rules as always - don't mindlessly copy the work, and make sure you understand how they got to the solution, not just what it does.  
Below is some demo code for `itertools` with $8$ items, but it could go arbitraily high to no ill-effect other than display and I/O issues.  

In [2]:
%%timeit
# demo of itertools permutation
count = 0
out = []
test = np.arange(8)+1
for j in itertools.permutations(test, len(test)):
    count += 1
    out.append(j)
#print(count)
#print(out)

3.28 ms ± 20 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In case it *doesn't* exist yet, or you can't find it, the next option is to make the simplest possible code work for at least a smaller version of the problem.  
I am **not** at this point asking you to do backtracking - you can if you want.  
All I am suggesting is that if you are truly stuck, *work incrementally* and at least make something that definitely works, speed and elegance be damned.  
We can always re-work it later when we're sure we have a good reference output, and you can always make it easier by scaling down the problem.   

When I think "reliable, if somewhat slow" I reach for `for`-loops.  
So below I have demo code for creating permutations of arrays with exactly length $5$.  
It should be clear from the structure that I made this incrementally from lower lengths, and can definitely (if painfully) be extended to length $8$.  

In [3]:
def perm_5(A):
    out = []
    n = len(A)
    if n != 5:
        raise ValueError("Need array length 5")

    for i in range(n):
        for j in range(n):
            if (j == i):
                continue
            for k in range(n):
                if (k == i) or (k == j):
                    continue
                for l in range(n):
                    if (l == i) or (l == j) or (l == k):
                        continue
                    for m in range(n):
                        if (m == i) or (m == j) or (m == k) or (m == l):
                            continue
                        perm = [A[i], A[j], A[k], A[l], A[m]]
                        out.append(perm)

    return out

test5 = np.arange(5)+1
%timeit perm_5(test5)
out = perm_5(test5)
print(len(out))
#out

82.6 µs ± 342 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
120


It's an order of magnitude slower than the native sort (for length 5), but it definitely works.  
The real downside is it only works for arrays of one specific length at a time, and it is incredibly inconvenient to change that length.  
At this point, I will ask you to look at the output, really stare at it for a while, and pick up on the pattern.  
You should notice that (due to the ordering of the loop and input) the left indices tend to be "locked" while the right indices swap around more frequently.  
This should guide you on how to generalize this to accept arbitrary array lengths.  

Now we can make a more general solution using backtracking, for which I have made demo code below.  
The idea is to make new solutions from the input by swapping elements in place (i.e. exactly how we would swap around rooks).  
Starting from the $0^{th}$ index, we try swapping in every available value to the right of it $(0-N)$ via the `for`-loop.  
For every swapped value in the $0^{th}$ spot, we recur on the $1^{st}$ index, trying every value to the right of it $(1-N)$; everything to the left is "locked in".  
Keep going until we reach the $N^{th}$ index, at which point we're done for that branch and we log the result.  
We then go back up the search tree by undoing the last swap we did - we can do this in code by applying the same line (a swap is its own inverse).  

In [4]:
def perm_bt(A):
    out = []
    
    def backtrack(ind):
        if ind == len(A): # current solution "cursor" index has reached max length, log it
            out.append(A.copy())
        for i in range(ind, len(A)): 
            A[ind], A[i] = A[i], A[ind] # Swap the elements
            backtrack(ind + 1)
            A[ind], A[i] = A[i], A[ind] # Backtrack by swapping again
            
    backtrack(0)
    return out

test = np.arange(5)+1
%timeit perm_bt(test)
permutations = perm_bt(test)
out = 0
for p in permutations:
    #print(p)
    out += 1
print(out)

174 µs ± 2.15 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
120


In [5]:
test = np.arange(8)+1
%timeit perm_bt(test)
permutations = perm_bt(test)
out = 0
for p in permutations:
    #print(p)
    out += 1
print(out)

59.7 ms ± 1.28 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
40320
[5 3 6 7 4 8 1 2]
[5 3 6 7 4 8 2 1]
[5 3 6 7 8 2 4 1]
[5 3 6 7 8 2 1 4]
[5 3 6 7 8 4 2 1]
[5 3 6 7 8 4 1 2]
[5 3 6 7 8 1 4 2]
[5 3 6 7 8 1 2 4]
[5 3 6 8 1 2 7 4]
[5 3 6 8 1 2 4 7]
[5 3 6 8 1 7 2 4]
[5 3 6 8 1 7 4 2]
[5 3 6 8 1 4 7 2]
[5 3 6 8 1 4 2 7]
[5 3 6 8 2 1 7 4]
[5 3 6 8 2 1 4 7]
[5 3 6 8 2 7 1 4]
[5 3 6 8 2 7 4 1]
[5 3 6 8 2 4 7 1]
[5 3 6 8 2 4 1 7]
[5 3 6 8 7 2 1 4]
[5 3 6 8 7 2 4 1]
[5 3 6 8 7 1 2 4]
[5 3 6 8 7 1 4 2]
[5 3 6 8 7 4 1 2]
[5 3 6 8 7 4 2 1]
[5 3 6 8 4 2 7 1]
[5 3 6 8 4 2 1 7]
[5 3 6 8 4 7 2 1]
[5 3 6 8 4 7 1 2]
[5 3 6 8 4 1 7 2]
[5 3 6 8 4 1 2 7]
[5 3 7 4 1 6 2 8]
[5 3 7 4 1 6 8 2]
[5 3 7 4 1 2 6 8]
[5 3 7 4 1 2 8 6]
[5 3 7 4 1 8 2 6]
[5 3 7 4 1 8 6 2]
[5 3 7 4 6 1 2 8]
[5 3 7 4 6 1 8 2]
[5 3 7 4 6 2 1 8]
[5 3 7 4 6 2 8 1]
[5 3 7 4 6 8 2 1]
[5 3 7 4 6 8 1 2]
[5 3 7 4 2 6 1 8]
[5 3 7 4 2 6 8 1]
[5 3 7 4 2 1 6 8]
[5 3 7 4 2 1 8 6]
[5 3 7 4 2 8 1 6]
[5 3 7 4 2 8 6 1]
[5 3 7 4 8 6 2 1]
[5 3 

[6 4 5 8 7 2 3 1]
[6 4 5 8 7 2 1 3]
[6 4 5 8 2 1 7 3]
[6 4 5 8 2 1 3 7]
[6 4 5 8 2 7 1 3]
[6 4 5 8 2 7 3 1]
[6 4 5 8 2 3 7 1]
[6 4 5 8 2 3 1 7]
[6 4 1 2 5 3 7 8]
[6 4 1 2 5 3 8 7]
[6 4 1 2 5 7 3 8]
[6 4 1 2 5 7 8 3]
[6 4 1 2 5 8 7 3]
[6 4 1 2 5 8 3 7]
[6 4 1 2 3 5 7 8]
[6 4 1 2 3 5 8 7]
[6 4 1 2 3 7 5 8]
[6 4 1 2 3 7 8 5]
[6 4 1 2 3 8 7 5]
[6 4 1 2 3 8 5 7]
[6 4 1 2 7 3 5 8]
[6 4 1 2 7 3 8 5]
[6 4 1 2 7 5 3 8]
[6 4 1 2 7 5 8 3]
[6 4 1 2 7 8 5 3]
[6 4 1 2 7 8 3 5]
[6 4 1 2 8 3 7 5]
[6 4 1 2 8 3 5 7]
[6 4 1 2 8 7 3 5]
[6 4 1 2 8 7 5 3]
[6 4 1 2 8 5 7 3]
[6 4 1 2 8 5 3 7]
[6 4 1 5 2 3 7 8]
[6 4 1 5 2 3 8 7]
[6 4 1 5 2 7 3 8]
[6 4 1 5 2 7 8 3]
[6 4 1 5 2 8 7 3]
[6 4 1 5 2 8 3 7]
[6 4 1 5 3 2 7 8]
[6 4 1 5 3 2 8 7]
[6 4 1 5 3 7 2 8]
[6 4 1 5 3 7 8 2]
[6 4 1 5 3 8 7 2]
[6 4 1 5 3 8 2 7]
[6 4 1 5 7 3 2 8]
[6 4 1 5 7 3 8 2]
[6 4 1 5 7 2 3 8]
[6 4 1 5 7 2 8 3]
[6 4 1 5 7 8 2 3]
[6 4 1 5 7 8 3 2]
[6 4 1 5 8 3 7 2]
[6 4 1 5 8 3 2 7]
[6 4 1 5 8 7 3 2]
[6 4 1 5 8 7 2 3]
[6 4 1 5 8 2 7 3]
[6 4 1 5 8

[7 5 4 1 3 2 6 8]
[7 5 4 1 3 2 8 6]
[7 5 4 1 3 8 2 6]
[7 5 4 1 3 8 6 2]
[7 5 4 1 8 6 3 2]
[7 5 4 1 8 6 2 3]
[7 5 4 1 8 3 6 2]
[7 5 4 1 8 3 2 6]
[7 5 4 1 8 2 3 6]
[7 5 4 1 8 2 6 3]
[7 5 4 8 2 6 1 3]
[7 5 4 8 2 6 3 1]
[7 5 4 8 2 1 6 3]
[7 5 4 8 2 1 3 6]
[7 5 4 8 2 3 1 6]
[7 5 4 8 2 3 6 1]
[7 5 4 8 6 2 1 3]
[7 5 4 8 6 2 3 1]
[7 5 4 8 6 1 2 3]
[7 5 4 8 6 1 3 2]
[7 5 4 8 6 3 1 2]
[7 5 4 8 6 3 2 1]
[7 5 4 8 1 6 2 3]
[7 5 4 8 1 6 3 2]
[7 5 4 8 1 2 6 3]
[7 5 4 8 1 2 3 6]
[7 5 4 8 1 3 2 6]
[7 5 4 8 1 3 6 2]
[7 5 4 8 3 6 1 2]
[7 5 4 8 3 6 2 1]
[7 5 4 8 3 1 6 2]
[7 5 4 8 3 1 2 6]
[7 5 4 8 3 2 1 6]
[7 5 4 8 3 2 6 1]
[7 5 2 4 3 6 1 8]
[7 5 2 4 3 6 8 1]
[7 5 2 4 3 1 6 8]
[7 5 2 4 3 1 8 6]
[7 5 2 4 3 8 1 6]
[7 5 2 4 3 8 6 1]
[7 5 2 4 6 3 1 8]
[7 5 2 4 6 3 8 1]
[7 5 2 4 6 1 3 8]
[7 5 2 4 6 1 8 3]
[7 5 2 4 6 8 1 3]
[7 5 2 4 6 8 3 1]
[7 5 2 4 1 6 3 8]
[7 5 2 4 1 6 8 3]
[7 5 2 4 1 3 6 8]
[7 5 2 4 1 3 8 6]
[7 5 2 4 1 8 3 6]
[7 5 2 4 1 8 6 3]
[7 5 2 4 8 6 1 3]
[7 5 2 4 8 6 3 1]
[7 5 2 4 8 1 6 3]
[7 5 2 4 8

[8 6 1 2 4 7 3 5]
[8 6 1 2 4 3 7 5]
[8 6 1 2 4 3 5 7]
[8 6 1 2 7 4 5 3]
[8 6 1 2 7 4 3 5]
[8 6 1 2 7 5 4 3]
[8 6 1 2 7 5 3 4]
[8 6 1 2 7 3 5 4]
[8 6 1 2 7 3 4 5]
[8 6 1 2 3 4 7 5]
[8 6 1 2 3 4 5 7]
[8 6 1 2 3 7 4 5]
[8 6 1 2 3 7 5 4]
[8 6 1 2 3 5 7 4]
[8 6 1 2 3 5 4 7]
[8 6 1 7 5 2 4 3]
[8 6 1 7 5 2 3 4]
[8 6 1 7 5 4 2 3]
[8 6 1 7 5 4 3 2]
[8 6 1 7 5 3 4 2]
[8 6 1 7 5 3 2 4]
[8 6 1 7 2 5 4 3]
[8 6 1 7 2 5 3 4]
[8 6 1 7 2 4 5 3]
[8 6 1 7 2 4 3 5]
[8 6 1 7 2 3 4 5]
[8 6 1 7 2 3 5 4]
[8 6 1 7 4 2 5 3]
[8 6 1 7 4 2 3 5]
[8 6 1 7 4 5 2 3]
[8 6 1 7 4 5 3 2]
[8 6 1 7 4 3 5 2]
[8 6 1 7 4 3 2 5]
[8 6 1 7 3 2 4 5]
[8 6 1 7 3 2 5 4]
[8 6 1 7 3 4 2 5]
[8 6 1 7 3 4 5 2]
[8 6 1 7 3 5 4 2]
[8 6 1 7 3 5 2 4]
[8 6 1 3 5 2 7 4]
[8 6 1 3 5 2 4 7]
[8 6 1 3 5 7 2 4]
[8 6 1 3 5 7 4 2]
[8 6 1 3 5 4 7 2]
[8 6 1 3 5 4 2 7]
[8 6 1 3 2 5 7 4]
[8 6 1 3 2 5 4 7]
[8 6 1 3 2 7 5 4]
[8 6 1 3 2 7 4 5]
[8 6 1 3 2 4 7 5]
[8 6 1 3 2 4 5 7]
[8 6 1 3 7 2 5 4]
[8 6 1 3 7 2 4 5]
[8 6 1 3 7 5 2 4]
[8 6 1 3 7 5 4 2]
[8 6 1 3 7

In terms of speed, it's 2-3 times slower than the explicit `for`-loop (length 5), which is expected for Python.  
But I would much rather spend a few milliseconds waiting for this code to run than recode a `for`-loop to nesting level 8.  
It's also about 2 orders of magnitude slower than `itertools`, which is pretty reasonable for basic Python code.  
Still (once again) if you need something fast, go with libraries and low-level code.  

## $N$-Queens

For the $N$-queens problem, it's a bit of a mixed bag in difficulty.  
The bad news is that we really have no exact formula for this (yet), just computer counts.   
The good news is that we can take a lot of our work for the rooks problem and apply it here: 
- There can only be one queen per rank/file, so we can use the same array-based representation for the queen positions. 
- Every solution to $N$-queens must satisfy $N$-rooks, so at worst we'll have to search all $N!$ rook positions. 

Of course as part of testing the rook positions, we need to rule out those with diagonal attacks.  
I am *not* going to tell you how to do this, I am sure you can figure it out on your own (hint: use computer arithmetic).  

Assuming you did that, question: should it be faster to just test the rook positions, or to just do a backtracking of the queens directly?  
Intuitively it has to be the latter - queens attack more squares than rooks, so it automatically excludes more possibilities as you backtrack.  
So if I put a queen in a corner square, backtracking won't bother putting the rest in any of the 7 other diagonals (whereas the rook problem has to check).  

The catch with backtracking here is that you are always guaranteed to be able to place $N$-rooks.  
For backtracking on $N$-queens, you may run out of space before placing all $N$ of them, so you need to account for that.  
Actually, you can avoid this problem altogether by generating the rook permutations, but immediately backtracking as soon as you hit a diagonal attack.  
This guarantees putting $N$ queens on the board, and cuts the search time down drastically.  
We will explore this in the next section on pruning.  

**Exercise:** Find the counts of $N$-queens for arbitrary $N$ (at least 8, but go as high as possible).  
Use whatever method you want, as long as it runs in reasonable time, and only uses standard Python and numpy. 

# Example - Sudoku (Pruning)

**Sudoku** is a game, typically played on a $9\times 9$ grid made of $3\times 3$ blocks, with some numbers pre-placed on the grid.  
The goal is to fill up every square s.t. every row, column, and $3\times 3$ block contains the numbers $1$ to $9$ exactly once.  
Skiena gives one example in Figure 7.2, which has 17 numbers as the starting position (the minimum needed for a unique solution to exist).  

To solve this with backtracking, simply pick an open spot and fill it in with a valid guess.  
Keep doing this until you either run out of valid options, or run into a contradiction (depending on how you implement the solver).  
Below I have some demo code I [found online](https://www.geeksforgeeks.org/sudoku-backtracking-7/) that gives the most naive approach to backtracking on Sudoku.  

In [6]:
test = [
    [0, 0, 0, 0, 0, 0, 0, 1, 2],
    [0, 0, 0, 0, 3, 5, 0, 0, 0],
    [0, 0, 0, 6, 0, 0, 0, 7, 0],
    [7, 0, 0, 0, 0, 0, 3, 0, 0],
    [0, 0, 0, 4, 0, 0, 8, 0, 0],
    [1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 2, 0, 0, 0, 0],
    [0, 8, 0, 0, 0, 0, 0, 4, 0],
    [0, 5, 0, 0, 0, 0, 6, 0, 0]
       ]

soln = [
    [6, 7, 3, 8, 9, 4, 5, 1, 2],
    [9, 1, 2, 7, 3, 5, 4, 8, 6],
    [8, 4, 5, 6, 1, 2, 9, 7, 3],
    [7, 9, 8, 2, 6, 1, 3, 5, 4],
    [5, 2, 6, 4, 7, 3, 8, 9, 1],
    [1, 3, 4, 5, 8, 9, 2, 6, 7],
    [4, 6, 9, 1, 2, 8, 7, 3, 5],
    [2, 8, 7, 3, 5, 6, 1, 4, 9],
    [3, 5, 1, 9, 4, 7, 6, 2, 8]
       ]

In [7]:
# Function to check if it is safe to place num at mat[row][col]
def isSafe(mat, row, col, num):
    # Check if num exists in the row
    for x in range(9):
        if mat[row][x] == num:
            return False

    # Check if num exists in the col
    for x in range(9):
        if mat[x][col] == num:
            return False

    # Check if num exists in the 3x3 sub-matrix
    startRow = row - (row % 3)
    startCol = col - (col % 3)

    for i in range(3):
        for j in range(3):
            if mat[i + startRow][j + startCol] == num:
                return False

    return True

# Function to solve the Sudoku problem
def solveSudokuRec(mat, row, col):
    # base case: Reached nth column of the last row
    if row == 8 and col == 9:
        return True

    # If last column of the row go to the next row
    if col == 9:
        row += 1
        col = 0

    # If cell is already occupied then move forward
    if mat[row][col] != 0:
        return solveSudokuRec(mat, row, col + 1)

    for num in range(1, 10):
        # If it is safe to place num at current position
        if isSafe(mat, row, col, num):
            mat[row][col] = num
            if solveSudokuRec(mat, row, col + 1):
                return True
            mat[row][col] = 0

    return False

In [8]:
%time solveSudokuRec(test, 0, 0)
np.array_equal(test, soln)

CPU times: user 33.7 s, sys: 8.16 ms, total: 33.7 s
Wall time: 33.9 s


True

Now, I hope it is abundantly clear to you, but this particular approach is both incredibly slow (for a computer) and needlessly inefficent.  
This is because when we solve this ourselves, we use reduction techniques to speed things up.  
To port that behavior over to our computers, we need to pick our squares more carefully.  
If we are literally guessing at every stage, we should start with the cells that have the fewest options.  
That way, even if we're very unlucky, we will only fail a few times before we guarantee getting the right number. 

<img src="img/17_sudoku_pot.png" style="width: 30em" />  

Above I have taken Skiena's example and just filled in the blanks with all potential values based on neighbors.  
I highlighted the most constrained cell (G2) in green for pure guessing.  
This cell should ultimately contain a 4, but even if we guessed 9, that is the only time we will ever be wrong.  
Compared to C3, we could be wrong 6 times before getting the right value.  
In other words, the former is cutting the search space(tree) in half while the latter is cutting it into 7ths.  
**N.b.** This should remind you of the binary search strategy from the first sessions.  

Skiena also goes over allocations of values (local count vs. look ahead).  
I will instead point out that we already know cell I3 (in cyan) has to be a 3 based on the requirement that the blocks contain all numbers.  
So that cell is actually maximally constrained already, regardless of the direct constraint by its neighbors.  
This is fantastic for us, because it means we can eliminate the 3 from the potential values in the neighboring cells (orange).  
Ergo, we have to make 6 fewer decisions in the future.  

<img src="img/17_sudoku_prune.png" style="width: 30em" />  

**Exercise:** Either write your own, or modify the sudoku solver above to prefer the most constrained squares (directly and indirectly).  
You may use the excel sheet with my solution to the board for references of safe moves, but note there may be some optional and non-optional move-orders.  
**Sidenote:** When I wrote the solutions to this board, it turned *all* of my steps were based on some completion argument.  
Not all boards seem to have this convenient property, but it guarantees your best result should be exactly 64 steps.    

Just to hammer it in: being right / ruling out wrong choices **early** is important to speeding up the work.  
That's because in the process of solving, we fill in more cells which increasingly reduces the space of potential values in blank ones.   
If our guess was wrong, we need to wait for a contradiction to show up, then undo all of our work.  
If our guess was right, we don't need to do any of that and all of the reductions will be **permanent**.  

In terms of the search tree, remember the tree height goes as the log of the number of nodes.  
Getting things right, or ruling out what is wrong earlier corresponds to cutting off high nodes s.t. their lower branches are never explored.  
This is the concept of *pruning*.  
Whereas a naive brute force algorithm may search through millions of wrong positions, a pruned search can avoid wrong positions entirely (64-move solution).  

Even in situations where pruning doesn't get everything, it is still a massive improvement.  
This is the operating principle behind [alpha-beta pruning](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) in chess engines.  
Chess, for now, is still hoplessly unsolvable and it would be implausible to exhaustively search all legal games.  
But in practice, we only really need to search to a certain depth to come up with a good move.  
So what chess engines (or at least some of them) do is [iterative deepening](https://en.wikipedia.org/wiki/Iterative_deepening_depth-first_search):
1. Run a depth-limited search, and return the best result. 
1. Prune some of the branches. 
1. Increase the depth and re-run. 

This necessarily re-examines old positions (say from the depth 10 search) in deeper searches (say depth 12), which seems inefficient.  
But the deeper searches improve the evaluation accuracy, which lets us safely prune even more branches at previous depths (higher on the tree).   
**Moral of the story:** Prune early and often.     

# Ending Notes

## Games
The reason I'm bringing up games rather than actual problems (like truck routes or warehouse management) is twofold.  
1. **Availability**  
It's pretty hard to get realistic material for actual business problems - companies aren't exactly keen to share data with a random yahoo on the internet.  
It only gets harder if you want to tackle problems that need combinatorial search, because the volume blows up (I'll leave that to your big data class).  
Most games are enjoyable enough that someone probably made a site you can just visit and start messing around without delay.  
In the interest of self-development, I want you to start looking for things on your own.  
This just illustrates that material is more ubiquitous than first appears, if you are inclined to look.  
1. **Difficulty**  
These are far simpler systems, and yet they aren't exactly easy; chess is not solved, and neither is $N$-queens.  
Real systems are hampered by things like noise, friction, irrationality, AND they tend to be of larger scale.   
If we can't nail down the ideal scenarios of games, we'll be utterly hopeless in the real world.  
If nothing else, games provide an easy way to come up with and test new heuristics.  

Chess is a particularly good example because it is complex enough to represent multiple problems.  
For example, the [lichess training program for piece movement](https://lichess.org/learn#/5/6) is exactly a traveling salesman problem.  

Regardless of how you feel about games, I will repeat this point: the single most important thing this class should impart to you is the power of *translation*.  
As long as you can recognize that a hard problem (truck route optimization) is related to an easier problem (chess movements), you have a starting point.  
The problems may differ by some noise or other non-trivial modifications, but at least you don't do everything from scratch.  
Or you might realize that your problem is functionally equivalent to an intractable one (see last topic), and know for a fact there are no convenient solutions.  

## Exact Cover
The problems of Sudoku and N-queens are examples of what is known as the [exact cover problem](https://en.wikipedia.org/wiki/Exact_cover#Detailed_example).  
Games aside, it clearly has applications to managing schedules (among other things).  
This problem has an associated algorithm to solve it, [Algorithm X](https://en.wikipedia.org/wiki/Knuth%27s_Algorithm_X), coming from none other than Donald Knuth himself.  

A direct implementation is tricky for us because he used circular doubly linked lists, and [dancing links](https://en.wikipedia.org/wiki/Dancing_Links), while we're using Python.  
But amazingly, someone (Ali Assaf) found out you can do basically the same thing with Python dictionaries in [exactly 30 lines of code](https://www.cs.mcgill.ca/~aassaf9/python/algorithm_x.html).  
Below I put a copy of his code in case his site ever goes down, but it is quite elegant.  


In [9]:
# Credit: Ali Assaf 

def solve(X, Y, solution=[]):
    if not X:
        yield list(solution)
    else:
        c = min(X, key=lambda c: len(X[c]))
        for r in list(X[c]):
            solution.append(r)
            cols = select(X, Y, r)
            for s in solve(X, Y, solution):
                yield s
            deselect(X, Y, r, cols)
            solution.pop()

def select(X, Y, r):
    cols = []
    for j in Y[r]:
        for i in X[j]:
            for k in Y[i]:
                if k != j:
                    X[k].remove(i)
        cols.append(X.pop(j))
    return cols

def deselect(X, Y, r, cols):
    for j in reversed(Y[r]):
        X[j] = cols.pop()
        for i in X[j]:
            for k in Y[i]:
                if k != j:
                    X[k].add(i)

In [10]:
#!/usr/bin/env python3

# Author: Ali Assaf <ali.assaf.mail@gmail.com>
# Copyright: (C) 2010 Ali Assaf
# License: GNU General Public License <http://www.gnu.org/licenses/>

from itertools import product

def solve_sudoku(size, grid):
    """ An efficient Sudoku solver using Algorithm X.

    >>> grid = [
    ...     [5, 3, 0, 0, 7, 0, 0, 0, 0],
    ...     [6, 0, 0, 1, 9, 5, 0, 0, 0],
    ...     [0, 9, 8, 0, 0, 0, 0, 6, 0],
    ...     [8, 0, 0, 0, 6, 0, 0, 0, 3],
    ...     [4, 0, 0, 8, 0, 3, 0, 0, 1],
    ...     [7, 0, 0, 0, 2, 0, 0, 0, 6],
    ...     [0, 6, 0, 0, 0, 0, 2, 8, 0],
    ...     [0, 0, 0, 4, 1, 9, 0, 0, 5],
    ...     [0, 0, 0, 0, 8, 0, 0, 7, 9]]
    >>> for solution in solve_sudoku((3, 3), grid):
    ...     print(*solution, sep='\\n')
    [5, 3, 4, 6, 7, 8, 9, 1, 2]
    [6, 7, 2, 1, 9, 5, 3, 4, 8]
    [1, 9, 8, 3, 4, 2, 5, 6, 7]
    [8, 5, 9, 7, 6, 1, 4, 2, 3]
    [4, 2, 6, 8, 5, 3, 7, 9, 1]
    [7, 1, 3, 9, 2, 4, 8, 5, 6]
    [9, 6, 1, 5, 3, 7, 2, 8, 4]
    [2, 8, 7, 4, 1, 9, 6, 3, 5]
    [3, 4, 5, 2, 8, 6, 1, 7, 9]
    """
    R, C = size
    N = R * C
    X = ([("rc", rc) for rc in product(range(N), range(N))] +
         [("rn", rn) for rn in product(range(N), range(1, N + 1))] +
         [("cn", cn) for cn in product(range(N), range(1, N + 1))] +
         [("bn", bn) for bn in product(range(N), range(1, N + 1))])
    Y = dict()
    for r, c, n in product(range(N), range(N), range(1, N + 1)):
        b = (r // R) * R + (c // C) # Box number
        Y[(r, c, n)] = [
            ("rc", (r, c)),
            ("rn", (r, n)),
            ("cn", (c, n)),
            ("bn", (b, n))]
    X, Y = exact_cover(X, Y)
    for i, row in enumerate(grid):
        for j, n in enumerate(row):
            if n:
                select(X, Y, (i, j, n))
    for solution in solve(X, Y, []):
        for (r, c, n) in solution:
            grid[r][c] = n
        yield grid

def exact_cover(X, Y):
    X = {j: set() for j in X}
    for i, row in Y.items():
        for j in row:
            X[j].add(i)
    return X, Y

def solve(X, Y, solution):
    if not X:
        yield list(solution)
    else:
        c = min(X, key=lambda c: len(X[c]))
        for r in list(X[c]):
            solution.append(r)
            cols = select(X, Y, r)
            for s in solve(X, Y, solution):
                yield s
            deselect(X, Y, r, cols)
            solution.pop()

def select(X, Y, r):
    cols = []
    for j in Y[r]:
        for i in X[j]:
            for k in Y[i]:
                if k != j:
                    X[k].remove(i)
        cols.append(X.pop(j))
    return cols

def deselect(X, Y, r, cols):
    for j in reversed(Y[r]):
        X[j] = cols.pop()
        for i in X[j]:
            for k in Y[i]:
                if k != j:
                    X[k].add(i)

#if __name__ == "__main__":
#    import doctest
#    doctest.testmod()